In [1]:
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import KTOTrainer, KTOConfig
from peft import LoraConfig, get_peft_model
import torch
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("C:\\Users\\CSE IIT BHILAI\\Desktop\\work\\pairs_for_pmponly.csv")

# Drop missing data
df = df.dropna(subset=["prompt", "response"]).reset_index(drop=True)

# Convert to required KTO columns
df["label"] = df["is_positive"].astype(bool)
df["completion"] = df["response"]

print(f"Total dataset size: {len(df)} rows")

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df["label"])
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(test_df)}")

# Convert to Hugging Face DatasetDict
train_ds = Dataset.from_pandas(train_df[["prompt", "completion", "label"]])
test_ds = Dataset.from_pandas(test_df[["prompt", "completion", "label"]])
dataset = DatasetDict({"train": train_ds, "test": test_ds})

print(dataset)
print(dataset["train"][0])

c:\Users\CSE IIT BHILAI\Desktop\work\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total dataset size: 17428 rows
Training samples: 12199
Validation samples: 5229
DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'label', '__index_level_0__'],
        num_rows: 12199
    })
    test: Dataset({
        features: ['prompt', 'completion', 'label', '__index_level_0__'],
        num_rows: 5229
    })
})
{'prompt': 'liar', 'completion': 'you are a liar', 'label': False, '__index_level_0__': 11963}


In [11]:
len(df)
df.columns

Index(['prompt', 'response', 'is_positive', 'src_idx', 'label', 'completion'], dtype='object')

In [2]:
# ============================================
# LOAD BASE MODEL & TOKENIZER
# ============================================

model_name = "EleutherAI/gpt-neo-1.3B"  # or "gpt2-medium", "gpt2-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # Important for KTO

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load policy model (will be trained)
policy_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
).to(device)

# Load reference model (frozen)
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
).to(device)

for param in ref_model.parameters():
    param.requires_grad = False

print("✅ Models loaded successfully")

c:\Users\CSE IIT BHILAI\Desktop\work\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\CSE IIT BHILAI\.cache\huggingface\hub\models--EleutherAI--gpt-neo-1.3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Models loaded successfully


In [3]:
# ============================================
# APPLY LoRA TO POLICY MODEL
# ============================================

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,                          # LoRA rank
    lora_alpha=32,                 # LoRA alpha (2*r)
    target_modules=[
        "c_attn",                  # GPT-2 attention
        "c_proj",                  # GPT-2 projection
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
policy_model = get_peft_model(policy_model, lora_config)
policy_model.print_trainable_parameters()

print("✅ LoRA applied to policy model")

trainable params: 3,932,160 || all params: 1,319,507,968 || trainable%: 0.2980
✅ LoRA applied to policy model


In [4]:
# ============================================
# CONFIGURE KTO TRAINING
# ============================================

from trl import KTOConfig

training_args = KTOConfig(
    output_dir="./kto-hh-rlhf-checkpoints",
    
    # Training hyperparameters
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-6,              # Lower than DPO
    num_train_epochs=3,
    
    # Optimization
    optim="adamw_torch",
    warmup_ratio=0.1,
    weight_decay=0.01,
    max_grad_norm=1.0,
    
    # Logging & Saving
    logging_steps=10,
    save_strategy="epoch",
    # evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=3,
    
    # KTO-specific parameters
    beta=0.1,                        # KTO temperature (like DPO)
    desirable_weight=1.0,            # Weight for good examples
    undesirable_weight=1.5,          # Weight for bad examples (λ > 1 for loss aversion!)
    
    # Sequence length
    max_length=512,
    max_prompt_length=256,
    max_completion_length=256,
    
    # Other
    report_to="none",
    remove_unused_columns=False,
    fp16=True if device == "cuda" else False,
)

print("✅ KTO Config created")
print(f"\n📊 Key KTO Parameters:")
print(f"  β (beta): {training_args.beta}")
print(f"  λ (loss aversion): {training_args.undesirable_weight / training_args.desirable_weight}")
print(f"  Desirable weight: {training_args.desirable_weight}")
print(f"  Undesirable weight: {training_args.undesirable_weight}")

✅ KTO Config created

📊 Key KTO Parameters:
  β (beta): 0.1
  λ (loss aversion): 1.5
  Desirable weight: 1.0
  Undesirable weight: 1.5


In [5]:
# ============================================
# INITIALIZE KTO TRAINER
# ============================================

trainer = KTOTrainer(
    model=policy_model,
    ref_model=ref_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
)

print("✅ Trainer initialized successfully!")
print(f"Training samples: {len(dataset['train'])}")
print(f"Eval samples: {len(dataset['test'])}")

c:\Users\CSE IIT BHILAI\Desktop\work\.venv\Lib\site-packages\trl\trainer\kto_trainer.py:359: UserWarning: This trainer will soon be moved to trl.experimental and is a candidate for removal. If you rely on it and want it to remain, please share your comments here: https://github.com/huggingface/trl/issues/4223. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  warnings.warn(
Extracting prompt from train dataset:   0%|          | 0/12199 [00:00<?, ? examples/s]

Processing tokenized eval KL dataset: 100%|██████████| 5229/5229 [00:01<00:00, 3439.26 examples/s]
You have different amounts of desirable/positive and undesirable/negative examples but the weights on the desirable and undesirable losses don't seem to be in an ideal range. Based on your data, we recommend EITHER desirable_weight in [1.33, 1.77] or undesirable_weight in [0.85, 1.12] (but NOT BOTH). See the documentation on how to optimally set these weights.


✅ Trainer initialized successfully!
Training samples: 12199
Eval samples: 5229


In [6]:
print("\n🚀 Starting KTO fine-tuning...")
train_result = trainer.train()

print("\n✅ Training complete!")
print(f"Training Loss: {train_result.training_loss:.4f}")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.



🚀 Starting KTO fine-tuning...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\CSE IIT BHILAI\Desktop\work\.venv\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.618800
20,0.614100
30,0.621900
40,0.615500
50,0.610900
60,0.609200
70,0.621800
80,0.612300
90,0.600000
100,0.620200


c:\Users\CSE IIT BHILAI\Desktop\work\.venv\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\CSE IIT BHILAI\Desktop\work\.venv\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



✅ Training complete!
Training Loss: 0.4637


In [13]:
import torch
from tqdm import tqdm
import numpy as np

def compute_perplexity(model, tokenizer, dataset, max_samples=5000):
    model.eval()
    losses = []

    for example in tqdm(dataset.select(range(min(max_samples, len(dataset)))), desc="Evaluating"):
        text = example["prompt"] + " " + example["completion"]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss.item()
        losses.append(loss)

    mean_loss = np.mean(losses)
    perplexity = np.exp(mean_loss)
    return perplexity

# Evaluate on validation split
ppl = compute_perplexity(policy_model, tokenizer, dataset["test"])
print(f"\n🧠 Perplexity on test set: {ppl:.3f}")


Evaluating: 100%|██████████| 5000/5000 [06:05<00:00, 13.69it/s]


🧠 Perplexity on test set: 17.386
